## Detector de carriles con en pista de atletismo Raspberry Pi 


Elaborado por: Luis Felipe Flores Zertuche   
Matrícula: 523197  
Carrera: ITR  
Fecha: 2019-05-23  


#### Introducción

Este proyecto consta de un carro controlado por una Raspberry Pi 3 B+ y que maneja de forma autonoma en una pista de atletismo. De hardware utiliza una Raspberry Camera v2 para la deteccion de lineas y un puente H para controlar los 4 motores y de software utiliza python3 junto con OpenCv para el procesamiento de imagenes.

A continuacion se explicara paso a paso como realizar este proyecto.

#### Objetivos 

* Diseñar y armar el carro con todos sus componentes.

* Controlar el carro con un control o teclado para que se mueve y utilice la camara.

* Realizar un video de la pista grabado del carro para poder realizar la deteccion de lineas en un video.

* Crear un algoritmo de control para que el carro detecte el centro del carril en el video.

* Conducir en la pista de atletismo de manera autonoma.

#### Procedimiento



- Instalar sistema operativo (Raspbian).
- Instalar librerías (open cv, RPi.GPIO, picamera).
- Recoleccion de datos de la pista de atletismo (Fotos y videos).
- Filtrado y obtencion de bordes.
- Obtener hough lines en una imagen.
- Metodo para el control autonomo del carro.
- Simulacion de metodo de control con un video.
- Lograr el movimiento autonomo en tiempo real del vehiculo.

##### Instalar Sistema Operativo

Raspbian es el sistema operativo oficial de Raspberry Pi.

El enlace oficial para descargar Raspbian:
https://www.raspberrypi.org/downloads/raspbian/

Para instalarlo seguir esta guia de instalacion:
https://www.raspberrypi.org/documentation/installation/installing-images/README.md

##### Instalar librerías

Las librerias principales que utilizamos en este proyecto son:
- OpenCv: Para el procesamiento de imagenes.
- RPI.GPIO: Para el control de las entradas y salidas (Motores).
- picamera: Para utilizar la Raspberry Camera.
- time: Para poder utilizar delays y calcular el tiempo que tarda en correr.
- math y numpy: Para realizar operaciones matematicas. 


##### Recoleccion de datos de la pista de atletismo (Fotos y videos)

Para poder empezar a filtrar y encontrar lineas en una imagen, primero tenemos que tener la imagen y para eso se crea el un programa para controlar el carro y tomar fotos y videos desde el teclado.

In [ ]:
"""
    Raspberry Controled car

    Control the car with arrow keys,
    Enter = Stop
    f = Take photo
    v = Start recording
    s = Stop recording

    Luis Felipe Flores Zertuche
    Universidad de Monterrey
"""

import RPi.GPIO as GPIO
import time
import curses
import picamera
from datetime import datetime

print('Raspberry Controled car ','\n','Control the car with arrow keys','\n')
print('Enter = Stop','\n','f = Take photo','\n','v = Start recording','\n','s = Stop recording')

# Create a time object, datetime.now() = date & time with microseconds
#time = datetime.now()

# Create a camera object from picamera
camera = picamera.PiCamera()
camera.resolution = (640,360)      # 1/3 of max resolution
camera.framerate = 20

# Get the curses window, turn off echoing of keyboard to screen, turn on
# instant (no waiting) key response, and use special values for cursor keys
screen = curses.initscr()
curses.noecho() 
curses.cbreak()
screen.keypad(True)

GPIO.setmode(GPIO.BCM)

motorLeftF = 5          # GPIO 5, move left wheels foward, IN1
motorLeftR = 6          # GPIO 6, move left wheels backward, IN2

motorRightF = 19        # GPIO 13, move right wheels foward, IN3
motorRightR = 13        # GPIO 19, move right wheels backward, IN4


GPIO.setup(motorLeftF, GPIO.OUT)
GPIO.setup(motorLeftR, GPIO.OUT)
GPIO.setup(motorRightF, GPIO.OUT)
GPIO.setup(motorRightR, GPIO.OUT)

def moveFoward():
    GPIO.output(motorLeftF, GPIO.LOW)
    GPIO.output(motorLeftR, GPIO.HIGH)
    GPIO.output(motorRightF, GPIO.LOW)
    GPIO.output(motorRightR, GPIO.HIGH)

def moveBackward():
    GPIO.output(motorLeftF, GPIO.HIGH)
    GPIO.output(motorLeftR, GPIO.LOW)
    GPIO.output(motorRightF, GPIO.HIGH)
    GPIO.output(motorRightR, GPIO.LOW)

def rotateRight():
    GPIO.output(motorLeftF, GPIO.HIGH)
    GPIO.output(motorLeftR, GPIO.HIGH)
    GPIO.output(motorRightF, GPIO.LOW)
    GPIO.output(motorRightR, GPIO.HIGH)

def rotateLeft():
    GPIO.output(motorLeftF, GPIO.LOW)
    GPIO.output(motorLeftR, GPIO.HIGH)
    GPIO.output(motorRightF, GPIO.HIGH)
    GPIO.output(motorRightR, GPIO.HIGH)

def rotateBLeft():
    GPIO.output(motorLeftF, GPIO.LOW)
    GPIO.output(motorLeftR, GPIO.LOW)
    GPIO.output(motorRightF, GPIO.HIGH)
    GPIO.output(motorRightR, GPIO.LOW)

def rotateBRight():
    GPIO.output(motorLeftF, GPIO.HIGH)
    GPIO.output(motorLeftR, GPIO.LOW)
    GPIO.output(motorRightF, GPIO.LOW)
    GPIO.output(motorRightR, GPIO.LOW)

def carStop():
    GPIO.output(motorLeftF, GPIO.LOW)
    GPIO.output(motorLeftR, GPIO.LOW)
    GPIO.output(motorRightF, GPIO.LOW)
    GPIO.output(motorRightR, GPIO.LOW)

try:
        while True:   
            char = screen.getch()
            if char == ord('q'):
                break
            elif char == curses.KEY_UP:
                moveFoward()
            elif char == curses.KEY_DOWN:
                moveBackward()
            elif char == curses.KEY_RIGHT:
                rotateRight()
            elif char == curses.KEY_LEFT:
                rotateLeft()
            elif char == 10:
                carStop()
            elif char == ord('k'):
                rotateBRight()
            elif char == ord('l'):
                rotateBLeft()
            elif char == ord('f'):
                camera.capture(str(datetime.now())+'.jpg')
                print('Snapshot saved')
            elif char == ord('v'):
                camera.start_recording(str(datetime.now()) +'.h264')
                print('Start recording, press S to stop')
            elif char == ord('s'):
                camera.stop_recording()
                print('Video saved')
             
finally:
    #Close down curses properly, inc turn echo back on!
    curses.nocbreak(); screen.keypad(0); curses.echo()
    curses.endwin()

GPIO.cleanup()

Raspberry Controled car  
 Control the car with arrow keys 

Enter = Stop 
 f = Take photo 
 v = Start recording 
 s = Stop recording


##### Filtrado y obtencion de bordes

Para que el programa detecte los bordes correctamente, la imagen tiene que pasar por tres filtros.


- cv2.cvtColor(img, cv2.COLOR_RGB2GRAY): Transforma la imagen a escala de grises.
- cv2.GaussianBlur(img_grey, kernel_size, sigmaX=0, sigmaY=0): Difumina la imagen.
- cv2.Canny(img_blur, low_threshold, high_threshold, apertureSize=3): Obtiene los bordes.

La pista de atletismo tiene muchas piedritas las cuales crean ruido a la hora de sacar los bordes (se ven bordes por todas partes). Para resolver esto aumente el kernel filtro Gaussian Blur para difuminar las piedras sin perder los bordes que nos interesan (los carriles).

In [ ]:
"""
    Filter and edge detection

    Luis Felipe Flores Zertuche
    Universidad de Monterrey
"""
# import required libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math


# run line detection pipeline
def run_pipeline(img_name):


    # 1.- Read image
    img_colour = cv2.imread(img_name)

    # verify that image `img` exist
    if img_colour is None:
        print('ERROR: image ', img_name, 'could not be read')
        exit()

    # 2. Convert from BGR to RGB then from RGB to greyscale
    img_colour_rgb = cv2.cvtColor(img_colour, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img_colour_rgb, cv2.COLOR_RGB2GRAY)

    # 3.- Apply Gaussuan smoothing
    kernel_size = (21,21)
    blur_gray = cv2.GaussianBlur(gray, kernel_size, sigmaX=0, sigmaY=0)

    #blur = cv2.bilateralFilter(grey, 21, 0, 0)

    # 4.- Apply Canny edge detector
    low_threshold = 60
    high_threshold = 80
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold, apertureSize=3)


    plt.figure(1)
    plt.imshow(gray, cmap='gray')
    plt.axis('off')

    plt.figure(2)
    plt.imshow(blur_gray, cmap='gray')
    plt.axis('off')

    plt.figure(3)
    plt.imshow(edges, cmap='gray')
    plt.axis('off')

    plt.show()
    return None

# Run pipeline
img_name = 'pista.jpg'
run_pipeline(img_name)

##### Obtener Hough lines en una imagen

Es mucho mas sencillo analizar una imagen que un video, por eso empezamos obteniendo las lineas de Hough en una imagen. Asi podemos verificar si los resultados obtenidos son los deseados.

In [ ]:
"""
    Line detection in an img using Hough lines

    Luis Felipe Flores Zertuche
    Universidad de Monterrey
"""

# import required libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math

# select a region of interest
def region_of_interest(img, vertices):
    """
    Applies an image mask.

    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)

    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    #filling pixels inside the polygon defined by "vertices" with the fill color
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image



# run line detection pipeline
def run_pipeline(img_name):


    # 1.- Read image
    img_colour = cv2.imread(img_name)

    # verify that image `img` exist
    if img_colour is None:
        print('ERROR: image ', img_name, 'could not be read')
        exit()

    # 2. Convert from BGR to RGB then from RGB to greyscale
    img_colour_rgb = cv2.cvtColor(img_colour, cv2.COLOR_BGR2RGB)
    grey = cv2.cvtColor(img_colour_rgb, cv2.COLOR_RGB2GRAY)

    # 3.- Apply Gaussuan smoothing
    kernel_size = (21,21)
    blur_grey = cv2.GaussianBlur(grey, kernel_size, sigmaX=0, sigmaY=0)

    # 4.- Apply Canny edge detector
    low_threshold = 60
    high_threshold = 80
    edges = cv2.Canny(blur_grey, low_threshold, high_threshold, apertureSize=3)

    # 5.- Define a polygon-shape like region of interest
    img_shape = grey.shape


    # Region of interest
    bottom_left = (0, 350)
    top_left = (0, 80)
    top_right = (1280, 80)
    bottom_right = (1280, 350)

    # create a vertices array that will be used for the roi
    vertices = np.array([[bottom_left,top_left, top_right, bottom_right]], dtype=np.int32)

    # 6.- Get a region of interest using the just created polygon. This will be
    #     used together with the Hough transform to obtain the estimated Hough lines
    masked_edges = region_of_interest(edges, vertices)

    # 7.- Apply Hough transform for lane lines detection
    rho = 1                       # distance resolution in pixels of the Hough grid
    theta = np.pi/180             # angular resolution in radians of the Hough grid
    threshold = 10                # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 5              # minimum number of pixels making up a line
    max_line_gap = 10              # maximum gap in pixels between connectable line segments
    line_image = np.copy(img_colour)*0   # creating a blank to draw lines on
    hough_lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)

    # 8.- Visualise input and output images
    img_colour_with_lines = img_colour_rgb.copy()
##    if(hough_lines !=None):
##        for line in hough_lines:
##            for x1, y1, x2, y2 in line:
##                cv2.line(img_colour_with_lines, (x1, y1), (x2, y2), (0,0,255), 3)
##                slope = (y2 - y1) / (x2 - x1) # <-- Calculating the slope.
##                print(slope)

    # 9.- Define left and right lane
    left_line_x = []
    left_line_y = []
    right_line_x = []
    right_line_y = []

    if(hough_lines != []):
        for line in hough_lines:
            for x1, y1, x2, y2 in line:
                cv2.line(img_colour_with_lines, (x1, y1), (x2, y2), (0,0,255), 3)
                slope = float(y2 - y1) / (x2 - x1) # <-- Calculating the slope.
                #print(slope)
                # if math.fabs(slope) < 0.5: # <-- Only consider extreme slope
                #     continue
                if slope <= 0: # <-- If the slope is negative, left group.
                    left_line_x.extend([x1, x2])
                    left_line_y.extend([y1, y2])
                else: # <-- Otherwise, right group.
                    right_line_x.extend([x1, x2])
                    right_line_y.extend([y1, y2])

    min_y = 0 # <-- Just below the horizon
    max_y = 230 # <-- The bottom of the image


    poly_left = np.poly1d(np.polyfit(
    left_line_y,
    left_line_x,
    deg=1
    ))

    left_x_start = int(poly_left(max_y))
    left_x_end = int(poly_left(min_y))

    poly_right = np.poly1d(np.polyfit(
    right_line_y,
    right_line_x,
    deg=1
    ))

    right_x_start = int(poly_right(max_y))
    right_x_end = int(poly_right(min_y))

    fullline = [[
                [left_x_start, max_y, left_x_end, min_y],
                [right_x_start, max_y, right_x_end, min_y],
                ]]
    
    #print(fullline)

    img_colour_with_fulllines = img_colour_rgb.copy()    
    for line in fullline:
    	for x1, y1, x2, y2 in line:
    		cv2.line(img_colour_with_fulllines, (x1, y1), (x2, y2), (0,255,0), 3)

    pf_bottom_left = (left_x_end , min_y)
    pf_top_left = (left_x_start, max_y)
    pf_top_right = (right_x_start, max_y)
    pf_bottom_right = (right_x_end, min_y)

    pf_vertices = np.array([[pf_bottom_left,pf_top_left, pf_top_right, pf_bottom_right]], dtype=np.int32)
    cv2.fillPoly(img_colour_with_fulllines, pf_vertices, (255,0,0))


    # visualise input and output images
##    plt.figure(1)
##    plt.imshow(masked_edges)
##    plt.axis('off')
##
##    plt.figure(2)
##    plt.imshow(blur_grey, cmap='gray')
##    plt.axis('off')
##
##    plt.figure(3)
##    plt.imshow(edges, cmap='gray')  
##    plt.axis('off')

##    plt.figure(4)
##    plt.imshow(img_colour_with_lines)
##    plt.axis('off')

    plt.figure(5)
    plt.imshow(img_colour_with_fulllines, cmap='gray')
    plt.axis('off')

    #print (hough_lines)
    print(max(left_line_y))
    print(max(right_line_y))


    plt.show()
    return None

# Run pipeline
img_name = 'pista.jpg'
run_pipeline(img_name)


##### Metodo para el control autonomo del carro

Esta parte fue todo un reto ya que la pista de atletismo esta muy grande para el carro. Los carriles estan separados por 1.22 metros, como la camara no tiene mucha altura los carriles que detecta estan aproximadamente a 1 metro. 
El primer metodo que intente fue encontrar el centro de la imagen y compararlo con el centro de las Hough lines, pero al analizar varios videos que grabe me di cuenta que no siempre detectaba la linea hasta el final. Esto me puede crear un error al decidir a que direccion ir. 

Por esa razon tuve que buscar otra manera de hacerlo. Al detectar las Hough lines en el video me di cuenta que casi siempre se detecta bien el punto inferior de las lineas por lo que opte en usar los putos inferiores de ambas lineas y encontrar el valor minimo de y de ambas. Si los valores son iguales el carro va centrado y si uno es mayor que el otro es que se esta moviendose hacia un lado.

Para enontrar los valores minimos de las Hough lines utilice la funcion max(<carril a analizar>), se utiliza el maximo porque los pixeles en y van de arriba hacia abajo. Ya teniendo ambos valores los compare y con el resultado decido en que direccion ir. Para que no este cambiando de sentido constantemente le agregue una tolerancia de 20 pixeles a cada lado.

##### Simulacion del metodo de control con un video

Antes de probarlo en tiempo real, realice una simulacion con un video de la pista de atletismo que grabe con el carro en la parte de obtencion de datos. Encuentro los fps a los que corre el video de salida y la direccion que debe tomar el carro.

In [ ]:
"""
	Line detection using Hough lines

	Luis Felipe Flores
	Universidad de Monterrey
"""

# import required libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
import time

start = time.time()

# select a region of interest
def region_of_interest(img, vertices):
    """
    Applies an image mask.

    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)

    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    #filling pixels inside the polygon defined by "vertices" with the fill color
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image



# run line detection pipeline
def run_pipeline():

    # initialise a video capture object
    cap = cv2.VideoCapture("Video_Pista2.h264")

    # check that the videocapture object was successfully created
    if(not cap.isOpened()):
        print("Error opening video source")
        exit()
    
    # create new windows for visualisation purposes
    cv2.namedWindow('input video', cv2.WINDOW_AUTOSIZE)

    while (cap.isOpened):
        
        flagstart = time.time()
        
        # grab current frame
        ret, frame = cap.read()        

        # verify that frame was properly captured
        if ret == False:
            print("ERROR: current frame could not be read")            
            break

        # 1.- Read image
        #frame_rs = cv2.resize(frame,(640,360))
        img_colour = frame


        # verify that image `img` exist
        if img_colour is None:
            print('ERROR: image ', img_name, 'could not be read')
            exit()

        # 2. Convert from BGR to RGB then from RGB to greyscale
        img_colour_rgb = img_colour
        grey = cv2.cvtColor(img_colour_rgb, cv2.COLOR_RGB2GRAY)

        # 3.- Apply Gaussuan smoothing
        kernel_size = (21,21)
        blur_grey = cv2.GaussianBlur(grey, kernel_size, sigmaX=0, sigmaY=0)

        # 4.- Apply Canny edge detector
        low_threshold = 60
        high_threshold = 80
        edges = cv2.Canny(blur_grey, low_threshold, high_threshold, apertureSize=3)

        # 5.- Define a polygon-shape like region of interest
        img_shape = grey.shape


        # Region of interest
        bottom_left = (0, 175)
        top_left = (0, 40)
        top_right = (640, 40)
        bottom_right = (640, 175)
        

        # create a vertices array that will be used for the roi
        vertices = np.array([[bottom_left,top_left, top_right, bottom_right]], dtype=np.int32)
        

        # 6.- Get a region of interest using the just created polygon. This will be
        #     used together with the Hough transform to obtain the estimated Hough lines
        masked_edges = region_of_interest(edges, vertices)

        # 7.- Apply Hough transform for lane lines detection
        rho = 1                       # distance resolution in pixels of the Hough grid
        theta = np.pi/180             # angular resolution in radians of the Hough grid
        threshold = 10                # minimum number of votes (intersections in Hough grid cell)
        min_line_len = 5              # minimum number of pixels making up a line
        max_line_gap = 10              # maximum gap in pixels between connectable line segments
        line_image = np.copy(img_colour)*0   # creating a blank to draw lines on
        hough_lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)

        # 8.- Visualise input and output images
        img_colour_with_lines = img_colour_rgb.copy()

        # 9.- Define left and right lane
        left_line_x = []
        left_line_y = []
        right_line_x = []
        right_line_y = []

        if(hough_lines !=None):
            for line in hough_lines:
                for x1, y1, x2, y2 in line:
                    cv2.line(img_colour_with_lines, (x1, y1), (x2, y2), (0,0,255), 3)
                    slope = float(y2 - y1) / (x2 - x1) # <-- Calculating the slope.
                    #print(slope)
                    if math.fabs(slope) < 0.3: # <-- Only consider extreme slope
                        continue
                    if slope <= 0: # <-- If the slope is negative, left group.
                        left_line_x.extend([x1, x2])
                        left_line_y.extend([y1, y2])
                    else: # <-- Otherwise, right group.
                        right_line_x.extend([x1, x2])
                        right_line_y.extend([y1, y2])

        min_y = 0 # <-- Just below the horizon
        #max_y = 350 # <-- The bottom of the image
        max_y = 175
        

        if(left_line_y != [] and left_line_x != []):
            poly_left = np.poly1d(np.polyfit(
                left_line_y,
                left_line_x,
                deg=1
                ))

        left_x_start = int(poly_left(max_y))
        left_x_end = int(poly_left(min_y))

        if(right_line_y != [] and right_line_x != []):
            poly_right = np.poly1d(np.polyfit(
                right_line_y,
                right_line_x,
                deg=1
                ))
            right_x_start = int(poly_right(max_y))
            right_x_end = int(poly_right(min_y))

        fullline = [[
                    [left_x_start, max_y, left_x_end, min_y],
                    [right_x_start, max_y, right_x_end, min_y],
                    ]]
        
        #print(fullline)

        img_colour_with_fulllines = img_colour_rgb.copy()    
        for line in fullline:
            for x1, y1, x2, y2 in line:
                    cv2.line(img_colour_with_fulllines, (x1, y1), (x2, y2), (0,255,0), 3)


        flagend = time.time()
        #print(' Run time: ')
        runTime = flagend - flagstart
        fps = 1/runTime
        #print(runTime)
        #print(' Fps: ')
        #print(fps)
        Fps = 'Fps = ' + str(fps)

        # Add FPS to img
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img_colour_with_fulllines,Fps,(0,20), font, 1,(255,255,255),2,cv2.LINE_AA)

        # Calculate min left and right point in y
        if(left_line_y !=[] and right_line_y !=[]):
            left_min = max(left_line_y)
            right_min = max(right_line_y)
##          print(left_min)
##          print(right_min)
            direction = 'left = ' + str(left_min) + ' Right = ' + str(right_min)
            #print(direction)

            if (left_min > (right_min + 20)):
                motor = 'right'
                #rotateRight()
                #time.sleep(0.1)
                #moveFoward()
            elif(right_min > (left_min + 20)):
                motor = 'left'
                #rotateLeft()
                #time.sleep(0.1)
                #moveFoward()
            else:
                #moveFoward()
                motor = 'foward'
            #print(motor)

            cv2.putText(img_colour_with_fulllines,motor,(0,60), font, 1,(255,255,255),2,cv2.LINE_AA)
        

        #visualise current frame
        cv2.imshow('input video',img_colour_with_fulllines)

        # Display the resulting frame
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.imwrite('carriles.png', img_colour_with_fulllines)
            cv2.imwrite('carriles_segmentados.png', img_colour_with_fulllines)
            break

           
    return None

# Run pipeline
run_pipeline()


##### Movimiento autonomo del vehiculo

Para lograr esto simplemente hay que combinar el codigo del contol de motores y el acceso a la Raspberrry camera con  programa anterior de la simulacion detector de lineas y asi logramos el movimiento autonomo en tiempo real del vehiculo.

In [ ]:
"""
	Self driving car using Raspberry Pi

	Luis Felipe Flores
	Universidad de Monterrey
"""

# import the necessary packages
from picamera.array import PiRGBArray
from picamera import PiCamera
import time
import cv2
import numpy as np
import RPi.GPIO as GPIO
import math
 
# initialize the camera and grab a reference to the raw camera capture
camera = PiCamera()
camera.resolution = (640, 360)
camera.framerate = 20
rawCapture = PiRGBArray(camera, size=(640, 360))
 
# allow the camera to warmup
time.sleep(0.1)

GPIO.setmode(GPIO.BCM)

motorLeftF = 5          # GPIO 5, move left wheels foward, IN1
motorLeftR = 6          # GPIO 6, move left wheels backward, IN2

motorRightF = 19        # GPIO 13, move right wheels foward, IN3
motorRightR = 13        # GPIO 19, move right wheels backward, IN4

GPIO.setwarnings(False)

GPIO.setup(motorLeftF, GPIO.OUT)
GPIO.setup(motorLeftR, GPIO.OUT)
GPIO.setup(motorRightF, GPIO.OUT)
GPIO.setup(motorRightR, GPIO.OUT)

def moveFoward():
    GPIO.output(motorLeftF, GPIO.LOW)
    GPIO.output(motorLeftR, GPIO.HIGH)
    GPIO.output(motorRightF, GPIO.LOW)
    GPIO.output(motorRightR, GPIO.HIGH)

def moveBackward():
    GPIO.output(motorLeftF, GPIO.HIGH)
    GPIO.output(motorLeftR, GPIO.LOW)
    GPIO.output(motorRightF, GPIO.HIGH)
    GPIO.output(motorRightR, GPIO.LOW)

def rotateRight():
    GPIO.output(motorLeftF, GPIO.HIGH)
    GPIO.output(motorLeftR, GPIO.HIGH)
    GPIO.output(motorRightF, GPIO.LOW)
    GPIO.output(motorRightR, GPIO.HIGH)

def rotateLeft():
    GPIO.output(motorLeftF, GPIO.LOW)
    GPIO.output(motorLeftR, GPIO.HIGH)
    GPIO.output(motorRightF, GPIO.HIGH)
    GPIO.output(motorRightR, GPIO.HIGH)

def rotateBLeft():
    GPIO.output(motorLeftF, GPIO.LOW)
    GPIO.output(motorLeftR, GPIO.LOW)
    GPIO.output(motorRightF, GPIO.HIGH)
    GPIO.output(motorRightR, GPIO.LOW)

def rotateBRight():
    GPIO.output(motorLeftF, GPIO.HIGH)
    GPIO.output(motorLeftR, GPIO.LOW)
    GPIO.output(motorRightF, GPIO.LOW)
    GPIO.output(motorRightR, GPIO.LOW)

def carStop():
    GPIO.output(motorLeftF, GPIO.LOW)
    GPIO.output(motorLeftR, GPIO.LOW)
    GPIO.output(motorRightF, GPIO.LOW)
    GPIO.output(motorRightR, GPIO.LOW)
    
# capture frames from the camera
for frame in camera.capture_continuous(rawCapture, format="bgr", use_video_port=True):
	# grab the raw NumPy array representing the image, then initialize the timestamp
	# and occupied/unoccupied text
        flagstart = time.time()
	
        image = frame.array
	
	# 1.- Read image
        #frame_rs = cv2.resize(frame,(640,360))
        img_colour = image


        # verify that image `img` exist
        if img_colour is None:
            print('ERROR: image ', img_name, 'could not be read')
            exit()

        # 2. Convert from BGR to RGB then from RGB to greyscale
        img_colour_rgb = img_colour
        grey = cv2.cvtColor(img_colour_rgb, cv2.COLOR_RGB2GRAY)

        # 3.- Apply Gaussuan smoothing
        kernel_size = (21,21)
        blur_grey = cv2.GaussianBlur(grey, kernel_size, sigmaX=0, sigmaY=0)

        # 4.- Apply Canny edge detector
        low_threshold = 60
        high_threshold = 80
        edges = cv2.Canny(blur_grey, low_threshold, high_threshold, apertureSize=3)

        # 5.- Define a polygon-shape like region of interest
        img_shape = grey.shape

        # 6.- Apply Hough transform for lane lines detection
        rho = 1                       # distance resolution in pixels of the Hough grid
        theta = np.pi/180             # angular resolution in radians of the Hough grid
        threshold = 10                # minimum number of votes (intersections in Hough grid cell)
        min_line_len = 5              # minimum number of pixels making up a line
        max_line_gap = 10              # maximum gap in pixels between connectable line segments
        line_image = np.copy(img_colour)*0   # creating a blank to draw lines on
        hough_lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)

        # 7.- Visualise input and output images
        img_colour_with_lines = img_colour_rgb.copy()

        # 8.- Define left and right lane
        left_line_x = []
        left_line_y = []
        right_line_x = []
        right_line_y = []

        if(hough_lines !=None):
            for line in hough_lines:
                for x1, y1, x2, y2 in line:
                    cv2.line(img_colour_with_lines, (x1, y1), (x2, y2), (0,0,255), 3)
                    slope = float(y2 - y1) / (x2 - x1) # <-- Calculating the slope.
                    #print(slope)
                    if math.fabs(slope) < 0.2: # <-- Only consider extreme slope
                        continue
                    if slope <= 0: # <-- If the slope is negative, left group.
                        left_line_x.extend([x1, x2])
                        left_line_y.extend([y1, y2])
                    else: # <-- Otherwise, right group.
                        right_line_x.extend([x1, x2])
                        right_line_y.extend([y1, y2])

        min_y = 0
        max_y = 175
        

        if(left_line_y != [] and left_line_x != []):
            poly_left = np.poly1d(np.polyfit(
                left_line_y,
                left_line_x,
                deg=1
                ))
            left_x_start = int(poly_left(max_y))
            left_x_end = int(poly_left(min_y))

        if(right_line_y != [] and right_line_x != []):
            poly_right = np.poly1d(np.polyfit(
                right_line_y,
                right_line_x,
                deg=1
                ))
            right_x_start = int(poly_right(max_y))
            right_x_end = int(poly_right(min_y))
            
        img_colour_with_fulllines = img_colour_rgb.copy()
        if (left_line_y != [] and left_line_x != [] and right_line_y != [] and right_line_x != []):
            fullline = [[
                        [left_x_start, max_y, left_x_end, min_y],
                        [right_x_start, max_y, right_x_end, min_y],
                        ]]
            
            #print(fullline)
    
            for line in fullline:
                for x1, y1, x2, y2 in line:
                        cv2.line(img_colour_with_fulllines, (x1, y1), (x2, y2), (0,255,0), 3)

        

        # Calculate min left and right point in y
        if(left_line_y !=[] and right_line_y !=[]):
            left_min = max(left_line_y)
            right_min = max(right_line_y)
##          print(left_min)
##          print(right_min)
            direction = 'left = ' + str(left_min) + ' Right = ' + str(right_min)
            #print(direction)
            
            if (left_min > (right_min + 20)):
                motor = 'right'
                rotateRight()
                time.sleep(0.1)
                moveFoward()
            elif(right_min > (left_min + 20)):
                motor = 'left'
                rotateLeft()
                time.sleep(0.1)
                moveFoward()
            else:
                moveFoward()
                motor = 'foward'
            #print(motor)

            #cv2.putText(img_colour_with_fulllines,motor,(0,60), font, 1,(255,255,255),2,cv2.LINE_AA)
        

        flagend = time.time()
        #print(' Run time: ')
        runTime = flagend - flagstart
        fps = 1/runTime
        #print(runTime)
        #print(' Fps: ')
        #print(fps)
        Fps = 'Fps = ' + str(fps)

        # Add FPS to img
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img_colour_with_fulllines,Fps,(0,20), font, 1,(255,255,255),2,cv2.LINE_AA)
        

        #visualise current frame
        cv2.imshow('input video',img_colour_with_fulllines)

        # Display the resulting frame
        if cv2.waitKey(1) & 0xFF == ord('q'):
            #cv2.imwrite('carriles.png', img_colour_with_fulllines)
            #cv2.imwrite('carriles_segmentados.png', img_colour_with_fulllines)
            carStop()
            break
 
	# clear the stream in preparation for the next frame
        rawCapture.truncate(0)
 

#### Conclusion

Este proyecto no fue nada facil ya que es muy largo ycomplicado. Para poder realizarlo tuve que hacer parte por parte y todos los programas separados para poder entender que me da cada cosa y verificar si es el resultado es el que busco. Tuve que ser muy organizado a la hora de codificar para poder hacer todo por partes y al final juntarlo sin problemas. Utilice casi todo lo visto en el semestre para realizar este proyecto, lo cual me gusto mucho ya que termine entendiendo perfectamente lo que esta pasando en cada parte de mi programa. Uno de los retos principales fue que la pista estaba muy grande y los carriles muy separados para las dimensiones del carro, tuve que hacer varios videos con diferente angulo en la camara para decidir el angulo en que funciona mejor el programa. Otro reto fue que un dia antes cuando realize mi segunda prueba satisfactoria del carro se me descompuso el puente H, de suerte tenia otro de respuesto y lo pude cambiar.

Este proyecto todavia tiene muchas areas de oportunidad, ya que actualmente no calcula la curvatura de la vuelta ni predice a donde girar. Tambien se le puede agregar una red neuronal para que maneje autonomamente con datos previamente capturados. Otra oportunidad es desplegar la camara en un servidor junto con un mapa y con la velocidad y aceleracion del vehiculo y permitir el control del carro desde el servidor.